In [18]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

Златински Кирил ПИ18-1 Номер 55 Вариант 5

In [25]:
df = pd.read_csv('data.csv')
df

,Age,Delivey No,Delivery No,Blood of Pressure,Heart Problem,Caesarian
0,22,1,Timely,High,apt,No
1,26,2,Timely,Normal,apt,Yes
2,26,2,Premature,Normal,apt,No
3,28,1,Timely,High,apt,No
4,22,2,Timely,Normal,apt,Yes
...,...,...,...,...,...,...
75,27,2,Premature,Normal,apt,No
76,33,4,Timely,Normal,apt,Yes
77,29,2,Premature,High,apt,Yes
78,25,1,Latecomer,Low,apt,Yes


Задача классификации (бинарной)

In [26]:
df.isna().sum()

Age                  0
Delivey No           0
Delivery No          0
Blood of Pressure    0
Heart Problem        0
Caesarian            0
dtype: int64

Пропущенных значений нет

Небольшая описательная статистика датафрейма

In [27]:
df.describe()

,Age,Delivey No
count,80.000000,80.000000
mean,27.687500,1.662500
std,5.017927,0.794662
min,17.000000,1.000000
25%,25.000000,1.000000
50%,27.000000,1.000000
75%,32.000000,2.000000
max,40.000000,4.000000


In [28]:
df.corr()

,Age,Delivey No
Age,1.00000,0.42716
Delivey No,0.42716,1.00000


Делаю дамми-переменные

In [12]:
df = pd.get_dummies(df, columns=['Delivery No', 'Blood of Pressure', 'Heart Problem'])
df

,Age,Delivey No,Caesarian,Delivery No_Latecomer,Delivery No_Premature,Delivery No_Timely,Blood of Pressure_High,Blood of Pressure_Low,Blood of Pressure_Normal,Blood of Pressure_low,Heart Problem_apt,Heart Problem_inept
0,22,1,No,0,0,1,1,0,0,0,1,0
1,26,2,Yes,0,0,1,0,0,1,0,1,0
2,26,2,No,0,1,0,0,0,1,0,1,0
3,28,1,No,0,0,1,1,0,0,0,1,0
4,22,2,Yes,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
75,27,2,No,0,1,0,0,0,1,0,1,0
76,33,4,Yes,0,0,1,0,0,1,0,1,0
77,29,2,Yes,0,1,0,1,0,0,0,1,0
78,25,1,Yes,1,0,0,0,1,0,0,1,0


In [24]:
df.corr()

,Age,Delivey No,Delivery No_Latecomer,Delivery No_Premature,Delivery No_Timely,Blood of Pressure_High,Blood of Pressure_Low,Blood of Pressure_Normal,Blood of Pressure_low,Heart Problem_apt,Heart Problem_inept
Age,1.000000,0.427160,-0.028725,0.014171,0.012043,0.047761,-0.053384,0.022561,-0.083200,-0.250485,0.250485
Delivey No,0.427160,1.000000,-0.010157,-0.126242,0.112873,-0.009139,-0.207838,0.205780,-0.094389,-0.200267,0.200267
Delivery No_Latecomer,-0.028725,-0.010157,1.000000,-0.269841,-0.604218,0.052926,0.140917,-0.213896,0.216587,-0.039448,0.039448
Delivery No_Premature,0.014171,-0.126242,-0.269841,1.000000,-0.604218,0.052926,0.069112,-0.091670,-0.058444,0.086787,-0.086787
Delivery No_Timely,0.012043,0.112873,-0.604218,-0.604218,1.000000,-0.087594,-0.173802,0.252861,-0.130866,-0.039173,0.039173
Blood of Pressure_High,0.047761,-0.009139,0.052926,0.052926,-0.087594,1.000000,-0.322219,-0.577350,-0.064957,-0.149071,0.149071
Blood of Pressure_Low,-0.053384,-0.207838,0.140917,0.069112,-0.173802,-0.322219,1.000000,-0.558100,-0.062791,-0.053090,0.053090
Blood of Pressure_Normal,0.022561,0.205780,-0.213896,-0.091670,0.252861,-0.577350,-0.558100,1.000000,-0.112509,0.206559,-0.206559
Blood of Pressure_low,-0.083200,-0.094389,0.216587,-0.058444,-0.130866,-0.064957,-0.062791,-0.112509,1.000000,-0.145248,0.145248
Heart Problem_apt,-0.250485,-0.200267,-0.039448,0.086787,-0.039173,-0.149071,-0.053090,0.206559,-0.145248,1.000000,-1.000000


Разбиение датасета на тест и трейн подмножества

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Caesarian', axis=1), df['Caesarian'], test_size=0.33)

In [14]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.9811320754716981
0.6666666666666666


Далее- PCA

In [17]:
for i in range(1,10):
    pca = PCA(n_components=i)
    df_new = pca.fit_transform(df.drop('Caesarian', axis=1))
    
    X_train, X_test, y_train, y_test = train_test_split(df_new, df['Caesarian'], test_size=0.33)
    
    clf = с()

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(i)
    print(clf.score(X_train, y_train))
    print(clf.score(X_test, y_test))
    print('__________________________')

1
0.9622641509433962
0.5925925925925926
__________________________
2
0.9811320754716981
0.37037037037037035
__________________________
3
1.0
0.5925925925925926
__________________________
4
0.9622641509433962
0.4444444444444444
__________________________
5
0.9622641509433962
0.48148148148148145
__________________________
6
0.9433962264150944
0.5555555555555556
__________________________
7
0.9811320754716981
0.4444444444444444
__________________________
8
0.9811320754716981
0.5555555555555556
__________________________
9
0.9622641509433962
0.5185185185185185
__________________________


Качество на тесте и на трейне упало, но зато должна была вырасти скорость обучения. Из за того, что данных слишком мало, из за переразбиения сильно варьируется качество модели.

In [19]:
start_time = time.time()
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
print("--- %s seconds ---" % (time.time() - start_time))

0.9622641509433962
0.5555555555555556
--- 0.10911083221435547 seconds ---


Выбираю 6 компонент, так как при большем количестве качество практически не растет, но при этом слишком быстро приближается к исходному количеству столбцов в датафрейме.

In [21]:
pca = PCA(n_components=6)
df_new = pca.fit_transform(df.drop('Caesarian', axis=1))  
X_train, X_test, y_train, y_test = train_test_split(df_new, df['Caesarian'], test_size=0.33)

start_time = time.time()
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
print("--- %s seconds ---" % (time.time() - start_time))

0.9811320754716981
0.6296296296296297
--- 0.0850071907043457 seconds ---


В данном случае изменение времени (выигрыш по времени) незначительно(ый), так как исходный датасет очень мал. Но все таки оно есть. Если бы мы перебирали на огромном гриде, было бы намного ощутимее. Выигрыш во времени одновременно приносит и проигрыш по качеству из за алгоритма работы опорных векторов. Для столь малых датасетов я бы работал без использования PCA. 

Из за того, что датасет очень мал, все сильно зависит от рандома. При каждом запуске меняются показатели качества метрики score. Вышло так, что с PCA и тест и трейн- лучше.